In [1]:
import json
from copy import deepcopy
import numpy as np
import random as rnd

In [2]:
res_weight = 0
res_volume = 0
res_price = 0
res_items = []

flg_stagnation = 0 
population_one = []
population_fitness = []
child_population = []

Получаем данные

In [3]:
file = '1.txt'
with open(file) as f:
    data = f.readlines()

items = [line.strip().split() for line in data]

items = [[int(float(j)) for j in i] for i in items]

weight_capacity=float(items[0][0])
volume_capacity=float(items[0][1])
items.pop(0)

print('weight: ', weight_capacity, '\nvolume: ', volume_capacity)
capacities=[weight_capacity, volume_capacity]

weight:  13000.0 
volume:  12.0


Функция приспособленности

In [4]:
def fitness(one, data):
    weight = 0
    volume = 0
    price = 0
    
    for i in range(len(one)):
        if one[i]:
            weight += data[i][0]
            volume += data[i][1]
            price += data[i][2]

    if weight > weight_capacity or volume > volume_capacity:
        price = 0
    return price

1.1 случайная генерация

In [5]:
def make_pop(data):
    arr = []
    for i in range(len(data)):
        arr.append(rnd.randint(0, 1))
    return arr

2.2 выбрать только 20% самых приспособленных особей

In [6]:
def crossover_filter(population, data):
    choosen = []
    selected = []

    for i in range(len(population)):
        selected.append(fitness(population[i], data))
    
    len_choosen = int(len(selected) * 0.2)
    select_sort = deepcopy(selected)
    select_sort.sort(reverse = True)
    select_sort = select_sort[:len_choosen]
    
    for i in select_sort:
        indx = selected.index(i)
        if indx:
            choosen.append(population[indx])
    return choosen

3.1 многоточечный с 3мя точками

In [7]:
def crossover(parent1, parent2):
    dots = []
    for i in range(0, 3):
        dots.append(rnd.randint(1, len(parent1)))
    dots.sort()
    
    dot_one1 = parent1[:dots[0]] + parent1[dots[1]:dots[2]]
    dot_one2 = parent2[dots[0]:dots[1]] + parent2[dots[2]:]
    
    dot_two1 = parent1[dots[0]:dots[1]] + parent1[dots[2]:]
    dot_two2 = parent2[:dots[0]] + parent2[dots[1]:dots[2]]
    
    child1 = dot_one1 + dot_one2
    child2 =  dot_two1 + dot_two2
    return child1, child2

4.1 инвертирование всех битов у 1 особ

In [8]:
def mutation(population):
    if len(population) == 0:
        return population

    i = rnd.randint(0, len(population) - 1)
    choosen = population[i]

    for j in range(len(choosen)):
        if choosen[j] == 1:
            choosen[j] = 0 
        else:
            choosen[j] = 1

    population[i] = choosen
    return population

5.1 замена не более 30% худших особей на потомков

In [9]:
def update_population(old_pop, new_pop, data):
    merged_pop_1 = []
    merged_pop_2 = []
    
    for i in range(len(old_pop)):
        merged_pop_1.append((fitness(old_pop[i], data), old_pop[i]))
        
    merged_pop_1 = sorted(merged_pop_1, key=lambda x: x[0], reverse=True)
    len_old_pop =int(len(merged_pop_1) * 0.7)
    merged_pop_1 = merged_pop_1[:len_old_pop]

    for j in range(len(new_pop)):
        merged_pop_2.append((fitness(new_pop[j], data), new_pop[j]))
        
    merged_pop_2 = sorted(merged_pop_2, key=lambda x: x[0], reverse=True)
    len_new_pop =int(len(merged_pop_2) * 0.3)
    merged_pop_2 = merged_pop_1[:len_new_pop]

    better_pop = merged_pop_1
    for i in merged_pop_2:
        better_pop.append(i)
        
    choosen = []
    for q in range(len(better_pop)):
        choosen.append(better_pop[q][1])
    return choosen

Находим наиболее приспособленных индивида и популяцию

In [10]:
def max_fit_one(population, data):
    selected = []

    for i in range(len(population)):
        selected.append((fitness(population[i], data), population[i]))
    max_fit = selected[0]
    
    for j in range(len(selected)):
        if selected[j][0] > max_fit[0]:
            max_fit = selected[j]
    return max_fit

In [11]:
def max_fit_pop(population, data):
    selected = []

    for i in range(len(population)):
        selected.append(fitness(population[i], data))

    max_fit = selected[0]
    for j in range(len(selected)): 
        if selected[j] > max_fit:
            max_fit = selected[j]
    return max_fit

Генетический алгоритм, последовательно вывызваем все предыдущие функции

In [12]:
for i in range(200):
    population_one.append(make_pop(items)) # генерируем первую популяцию

for i in range(len(population_one)):
    population_fitness.append(fitness(population_one[i], items))

max_fit = population_fitness[0]
for j in range(len(population_fitness)):
    if population_fitness[j] > max_fit:
        max_fit = population_fitness[j]
current_fit = max_fit
next_population = mutation(population_one)

for g in range(500):
    parents = crossover_filter(next_population, items) # отбор для скрещивания
    for i in range(len(parents) - 1):
        child = crossover(parents[i], parents[i + 1]) # скрещивание
        child_population.append(child[0])
        child_population.append(child[1])
    child_population = mutation(child_population) # мутация

    old_pop = next_population
    next_population = update_population(next_population, child_population, items) # новая популяция
    next_fit = max_fit_pop(next_population, items)  # оценка результата
        
    #проверяем,сильно ли изменилась приспособленность, если в 10 поколениях подряд минимлальные изменения - выходим
    if abs((current_fit - next_fit) / ((current_fit + next_fit) / 2)) * 100 < 0.1:
        if flg_stagnation > 10:
            if current_fit > next_fit:
                res = max_fit_one(old_pop, items)
                break
            else:
                res = max_fit_one(next_population, items)
                break
        else:
            flg_stagnation += 1
    else:
        flg_stagnation = 0
        current_fit = next_fit
    res = max_fit_one(next_population, items)

In [13]:
for i in range(len(res[1])):
    if res[1][i] > 0:
        res_items.append(i)
        res_weight += float(items[i][0])
        res_volume += float(items[i][1])
        res_price += float(items[i][2])

Структурируем результат и пишем в json

In [14]:
res = {
    'weight': res_weight,
    'volume': res_volume,
    'price': res_price,
    'items': res_items
}

print(res)

with open('gen_self.json', 'w') as outfile:
    json.dump(res, outfile)

{'weight': 12196.0, 'volume': 4.0, 'price': 3651.0, 'items': [4, 5, 13, 14, 15, 16, 17, 18, 22, 23, 24, 25, 27, 29]}
